In [1]:
!pip install -q h5py pyyaml

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 2s 0us/step


In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [5]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 9s - loss: 2.4552 - accuracy: 0.0938
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 442us/sample - loss: 1.1264 - accuracy: 0.6690 - val_loss: 0.6899 - val_accuracy: 0.7980
Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 0.5138 - accuracy: 0.9062
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 99us/sample - loss: 0.4213 - accuracy: 0.8830 - val_loss: 0.5580 - val_accuracy: 0.8120
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.3285 - accuracy: 0.9375
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 96us/sample - loss: 0.2885 - accuracy: 0.9250 - val_loss: 0.4822 - val_accuracy: 0.8490
Epoch 4/10
  32/1000 [..............................] - ETA: 0s - loss: 0.1545 - accuracy: 1.0000
Epoch 0

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(acc)

1000/1000 - 0s - loss: 2.3503 - accuracy: 0.0540
0.054


In [8]:
checkpoint_path

'training_1/cp.ckpt'

In [9]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(acc)

1000/1000 - 0s - loss: 0.4131 - accuracy: 0.8700
0.87


### 5エポックごとに保存

In [11]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    verbose=1,
    period=5)

model = create_model()

model.fit(train_images, train_labels, epochs=50,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/50
1000/1000 [==============================] - 0s 268us/sample - loss: 1.1768 - accuracy: 0.6680 - val_loss: 0.7519 - val_accuracy: 0.7660
Epoch 2/50
1000/1000 [==============================] - 0s 84us/sample - loss: 0.4269 - accuracy: 0.8810 - val_loss: 0.5692 - val_accuracy: 0.8290
Epoch 3/50
1000/1000 [==============================] - 0s 79us/sample - loss: 0.2939 - accuracy: 0.9250 - val_loss: 0.4937 - val_accuracy: 0.8420
Epoch 4/50
1000/1000 [==============================] - 0s 76us/sample - loss: 0.2177 - accuracy: 0.9520 - val_loss: 0.4555 - val_accuracy: 0.8640
Epoch 5/50
  32/1000 [..............................] - ETA: 0s - loss: 0.1804 - accuracy: 0.9688
Epoch 00005: saving model to training_2/cp-0005.ckpt
1000/1000 [==============================] - 0s 93us/sample - loss: 0.1598 - accuracy: 0.9620 - val_loss: 0.4301 - val_accuracy: 0.8600
Epoch 6/50
1000/1000 [==============================] - 0s 86us/sample - los

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [13]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(acc)

1000/1000 - 0s - loss: 0.4862 - accuracy: 0.8740
0.874
